::::
:::{thebe-button}
:::
::::

# Combine mean absolute errors


In [ ]:
from boilercv_docs.nbs import HIDE, init

paths = init()

from tomllib import loads

from matplotlib.figure import Figure
from matplotlib.pyplot import subplots
from pandas import Categorical, DataFrame, concat, read_hdf
from seaborn import color_palette, scatterplot

from boilercv.correlations import GROUPS, META_TOML
from boilercv.correlations.models import Metadata
from boilercv_docs.nbs import set_display_options
from boilercv_pipeline.experiments.e230920_subcool import EXP_TIMES, MAE
from boilercv_pipeline.experiments.e240215_plotting import cat10

FIGURES: list[Figure] = []
"""Notebook figures available for export."""
MAX_BETA_MAE = 1.1
"""Maximum mean absolute error of beta to plot."""
MAX_NUSSELT_MAE = 200
"""Maximum mean absolute error of nusselt to plot."""
SCALE = 1.6
"""Plot scale."""
SIZE = 20 * SCALE
"""Size of the tracks."""
FONT_SCALE = SCALE
"""Font scale for plotting."""
WIDTH_SCALE = 2.00
"""Width scale for plotting."""
HEIGHT_SCALE = 1.30
"""Height scale for plotting."""
CORRELATIONS_PALETTE = cat10
"""For plotting correlations."""

HIDE

In [ ]:
set_display_options(FONT_SCALE)
meta = Metadata.context_model_validate(
    obj=loads(META_TOML.read_text("utf-8") if META_TOML.exists() else ""),
    context=Metadata.get_context(),
)
mae = DataFrame(
    columns=[
        "sortable_correlation_key",
        "Correlation",
        "Group",
        "Dimensionless bubble diameter",
        "Nusselt number",
        "Subcooling (K)",
    ],
    data=(
        concat([
            read_hdf(
                (MAE / f"mae_{time.isoformat().replace(':', '-')}").with_suffix(".h5")
            )
            for time in EXP_TIMES
        ])
        .reset_index()
        .rename(
            columns={
                "Beta (MAE)": "Dimensionless bubble diameter",
                "Nusselt (MAE)": "Nusselt number",
                "subcooling": "Subcooling (K)",
            }
        )
        .assign(**{
            "sortable_correlation_key": lambda df: Categorical(
                df["Correlation"], categories=list(meta), ordered=True
            ),
            "Group": lambda df: df["sortable_correlation_key"].map(GROUPS),
            "Correlation": lambda df: df["sortable_correlation_key"].map({
                corr: meta[corr].name
                for corr in df["sortable_correlation_key"].unique()
            }),
        })
    ),
)
mae

In [ ]:
def plot(mae, hue, palette):
    """Plot mean absolute error."""
    figure, ax = subplots()
    ax.set_xlim(0, MAX_BETA_MAE)
    ax.set_ylim(0, MAX_NUSSELT_MAE)
    FIGURES.append(figure)
    scatterplot(
        data=mae,
        x="Dimensionless bubble diameter",
        y="Nusselt number",
        s=SIZE,
        hue=hue,
        palette=palette,
    )
    figure.set_figwidth(WIDTH_SCALE * figure.get_figwidth())
    figure.set_figheight(HEIGHT_SCALE * figure.get_figheight())

In [ ]:
plot(
    mae=mae.sort_values("sortable_correlation_key"),
    hue="Correlation",
    palette=CORRELATIONS_PALETTE,
)
plot(mae=mae, hue="Subcooling (K)", palette=color_palette("flare", as_cmap=True))
plot(mae=mae.sort_values("Group"), hue="Group", palette=CORRELATIONS_PALETTE)
HIDE